### import tools

In [7]:
#from library import package
import pandas as pd
import numpy as np
import rpy2.robjects as ro

#from file import function
from countrynamefix import countrynamefix
from rcp_ssp import clean_rcpssp_co2
from gas_coal import clean_gas_coal
from tong import clean_tong
from steel import process_steel
from aggregation import aggregate
from expectations import make_expectation_over_data
from plotting import plot_data

### data cleaning and aggregation

In [8]:
## Matching country names across all data files
tong_cntry_fix, scen_cntry_fix, coal_cntry_fix, gas_cntry_fix, steel_cntry_fix = countrynamefix()

## Data cleaning
scen_CO2_cntry = clean_rcpssp_co2()
coal_panel, coalCO2_panel, gas_panel = clean_gas_coal(coal_cntry_fix, gas_cntry_fix)
steel_panel = process_steel()
tong_country_sector_CO2 = clean_tong(tong_cntry_fix)

# format consid/commit emissions by type
r = ro.r
r['source']('create_consid.R');
mk_consid_commit = ro.globalenv['mk_consid_commit']

# get all the data, and fix formatting
cntry_rgn_data = aggregate()
drop_cntry = cntry_rgn_data.drop(columns = ['country', 'Unnamed: 0'])
rgn_data = drop_cntry.groupby(by = ['SSP', 'RCP', 'region']).sum().reset_index()
world_totals = rgn_data.drop(columns = 'region').groupby(by = ['SSP', 'RCP']).sum().reset_index()

# aggregate the data by region and country respectively, forming 2 expectations
expectation_rgn = make_expectation_over_data(cntry_rgn_data, 'region')
expectation_cntry = make_expectation_over_data(cntry_rgn_data, 'country')
expectation_world = make_expectation_over_data(cntry_rgn_data, ['SSP', 'RCP']).drop(columns = ['credence', 'posterior', 'p', 'Unnamed: 0'])

# create plots of emissions trajectories in each of the corresponding folders

names without matches written to a csv
manually corrected list of country names parsed
initial data inputs written to files with corrected country names


### data analysis

In [9]:
comm_conn = []

for col in expectation_world.columns:
    if 'commit+consid.' in col:
            comm_conn.append(col)

# get the considered and committed emissions per year till 2100
comm_cons_expect = expectation_world[comm_conn]

# add to get the cummulative value for each SSP/RCP
comm_cons_expect['Cummulative cons+comm emits till 2100'] = comm_cons_expect.sum(axis = 1)

# add the rcp and ssp data back to the df
comm_cons_expect[['SSP', 'RCP']] = expectation_world[['SSP', 'RCP']]

# export as csv to examine in google sheet
# https://docs.google.com/spreadsheets/d/13i20peVmWWZA0PBWackXJBXF5T3npF8BdomHOu-hkkk/edit#gid=1192873491
comm_cons_expect.to_csv('data/annual_global_comm_consid_total.csv')

In [10]:
gt_gem_commit = pd.DataFrame(electricity_commit.iloc[:,1:].sum(axis = 0) * .001).T

yr_cols = []
for col in gt_gem_commit.columns:
    if '2025' in col:
        yr_cols.append(col)
gt_gem_commit[yr_cols]

NameError: name 'electricity_commit' is not defined

In [ ]:
summed_tong = pd.DataFrame(tong_country_sector_CO2.iloc[:,1:].sum(axis = 0))
summed_tong.iloc[:7,:].sum()

### plotting

In [ ]:
# # plot the raw country and region level data
plot_data(rgn_data, 'raw', 'Planned, Committed, and Considered Emissions till 2100 ', 'data/plots/rgn_emissions_rcp_ssp_plots/')
plot_data(cntry_rgn_data, 'raw', 'Planned, Committed, and Considered Emissions till 2100 ', 'data/plots/cntry_emissions_rcp_ssp_plots/')

# plot the data aggregated over the entire world, still raw
plot_data(world_totals, 'raw', 'Planned, Committed, and Considered Emissions till 2100 ', 'data/plots/world_ssp_rcp_emissions/')

# plot the expectation across countries and regions, respectively
plot_data(expectation_rgn, 'raw', 'Total Expectable, Committed, and Considered Emissions till 2100 ', 'data/plots/rgn_exp_plots/')
plot_data(expectation_cntry, 'raw', 'Total Expectable, Committed, and Considered Emissions till 2100 ', 'data/plots/cntry_exp_plots/')